In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from src.data_processing import AA_KEYS, compute_pfm, compute_ic_position, compute_ic, get_mia
from src.utils import pkl_dump, pkl_load
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 140
import warnings
warnings.filterwarnings("ignore")
from tqdm.autonotebook import tqdm
pd.set_option("display.max_columns",999)
pd.set_option("display.max_rows",100)
DATADIR = '../output_xls/filtered_rank20/'

In [12]:
for l in range(8,13):
    for hla in hlas:
        files = [os.path.join(DATADIR,x) for x in os.listdir(DATADIR) if hla in x and f'{l}mer' in x]
        df = pd.concat([pd.read_csv(x, sep = '\t', skiprows=1,header=0) for x in files])
        df1 = df.query('EL_Rank<0.25')
        df1= df1.drop(index=df1.loc[df1['Peptide'].apply(lambda x: any([z not in AA_KEYS for z in x]))].index)
        if len(df1)==0:
            print(l,hla)
        # df5 = df.query('EL_Rank<0.5')
        # drop_idx5 = df5.loc[df5['Peptide'].apply(lambda x: any([z not in AA_KEYS for z in x]))].index
        # print(f'1: len\t{l}\t{hla}\t{len(df1.drop(index=drop_idx1).Peptide.values)}') 
        # print(f'5: len\t{l}\t{hla}\t{len(df5.drop(index=drop_idx5).Peptide.values)}')

KeyboardInterrupt: 

In [10]:
# Parallelize
from joblib import Parallel, delayed
from functools import partial
import multiprocessing
def get_anchors_ics(hla, length,
                    ranks = [0.1, 0.25, 0.5], DATADIR='../output_xls/filtered_rank20/',
                    how='shannon', seq_weighting=False, beta=100):
    files = [os.path.join(DATADIR, x) for x in os.listdir(DATADIR) if hla in x and f'{length}mer' in x]
    if len(files)==0:
        return 0
    df = pd.concat([pd.read_csv(x, sep = '\t', skiprows=1, header=0) for x in files])
    res = {}
    res['hla'] = hla
    res['l'] = length
    for k in ranks:
        tmp = df.query('EL_Rank < @k')
       
        drop_idx = tmp.loc[tmp['Peptide'].apply(lambda x: any([z not in AA_KEYS for z in x]))].index
        seqs = tmp.drop(index=drop_idx).Peptide.values
        if len(seqs)==0:
            print(hla, length, k)
        res[k] = compute_ic(seqs, how, seq_weighting, beta)
    return res

In [12]:
hlas = np.unique([x.split('_')[0] for x in os.listdir(DATADIR) if 'chunk' in x and x.endswith('rank20.txt')])
lens = list(range(8,13))
for how in tqdm(['shannon', 'kl'], desc='how', leave=True):
    res = {}
    for l in tqdm(lens, desc='len', leave=True):
        res[l] = {}
        get_anchors_ics_ = partial(get_anchors_ics, length=l,
                                   ranks=[0.2, 0.25, 0.5], how=how,
                                   seq_weighting=False)
        output = Parallel(n_jobs=8)(delayed(get_anchors_ics_)(hla) for hla in hlas)
        for item in output:
            res[l][item['hla']]={}
            res[l][item['hla']][0.15] = item[0.15]
            res[l][item['hla']][0.25] = item[0.25]
            res[l][item['hla']][0.5] = item[0.5]
    pkl_dump(res, f'ics_{how}.pkl', '../output') 

how:   0%|          | 0/2 [00:00<?, ?it/s]

len:   0%|          | 0/5 [00:00<?, ?it/s]

Exception: []